In [2]:
import pandas as pd
import numpy as np
import requests
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [59]:
def percent(x, y):
    return (x/y)*100

# Selecting LEHD datasets: 

https://lehd.ces.census.gov/data/lodes/LODES7/LODESTechDoc7.5.pdf  

Pages 5 & 6 of this document outlines the file structure for Origin-Destination data. This data is limited to disaggregations by the following categories:  
+ age (29 and younger, 30 to 54, and 55 or older)  
+ earnings (1,250 a month or less, 1,251 to 3,333 a month, and greater than 3,333 a month)  
+ Goods-Producing versus Trade, Transportation & Utilities, versus All Other Services industry sectors  

The earnings breakout is kind of useless so we'll only use the age and industry sector data.  

Page 7 of this document outlines the file structure for the Residence Area Characteristics data. This shows the same totals for age and earnings listed above, but further disaggregates the industry component into 2-digit NAICS codes. It also disaggregates job counts by race, Hispanic or Latino ethnicity, sex, and educational attainment (High School or Equivalent, Some college or Associate degree, Bachelor's degree or advanced degree). 

Page 8 of this document outlines the file structure for the Workplace Area Characteristics. This shows all of the same information as the RAC but also firm age and size. 

This data package also has a geography crosswalk which will be helpful in aggregating all of these numbers to a place and county level.


In [138]:
#all primary jobs
url = 'https://lehd.ces.census.gov/data/lodes/LODES7/tn/rac/tn_rac_S000_JT01_2019.csv.gz'
rac = pd.read_csv(url)
print('Your file is ready.')

Your file is ready.


In [139]:
raccols = {'h_geocode': 'GEO_ID', 'C000': 'Total Jobs', 'CA01': 'Age:Workers 29 or Younger', 'CA02': 'Age:Workers 30 to 54', 
           'CA03': 'Age:Workers 55 Older', 'CE01': 'Earnings:1250mo or Less', 'CE02': 'Earnings:1251 to 3333mo', 
           'CE03': 'Earnings:Greater Than 3333mo', 'CNS01': 'NAICS:11Agriculture, Forestry, Fishing & Hunting', 
           'CNS02': 'NAICS:21Mining, Quarrying, Oil & Gas', 'CNS03': 'NAICS:22Utilities', 'CNS04': 'NAICS:23Construction', 'CNS05':'NAICS:31to33Manufacturing',
           'CNS06': 'NAICS:42Wholesale Trade', 'CNS07': 'NAICS:44to45Retail Trade', 'CNS08': 'NAICS:48to49Transportation & Warehousing',
           'CNS09': 'NAICS:51Information', 'CNS10': 'NAICS:52Finance & Insurance', 'CNS11': 'NAICS:53Real Estate & Rental & Leasing', 
           'CNS12': 'NAICS:54Professional, Scientific, & Technical Services', 'CNS13': 'NAICS:55Management of Companies & Enterprises', 
           'CNS14': 'NAICS:56Administrative & Support & Waste Management & Remediation Services', 'CNS15': 'NAICS:61Educational Services', 
           'CNS16': 'NAICS:62Healthcare & Social Assistance', 'CNS17': 'NAICS:71Arts, Entertainment, & Recreation', 'CNS18': 'NAICS:72Accommodation & Food Services', 
           'CNS19': 'NAICS:81Other Services (Except Public Administration)', 'CNS20': 'NAICS:92Public Administration', 
           'CR01': 'Race:White Alone', 'CR02': 'Race:Black or African American Alone', 'CR03': 'Race:American Indian or Alaska Native Alone', 
           'CR04': 'Race:Asian Alone', 'CR05': 'Race:Native Hawaiian or Other Pacific Islander Alone', 'CR07': 'Race:Two or More Races', 
           'CT01': 'Ethnicity:Not Hispanic or Latino', 'CT02': 'Ethnicity:Hispanic or Latino', 'CD01':'Ed:Less than High School', 
           'CD02': 'Ed:High School or Equivalent', 'CD03': 'Ed:Some College or Associates', 'CD04': 'Ed:Bachelors or Advanced Degree', 'CS01': 'Sex:Male', 
           'CS02': 'Sex:Female'}

In [140]:
rac.rename(columns = raccols, inplace = True)

In [141]:
rac['Age%:Workers 29 or Younger'] = percent(rac['Age:Workers 29 or Younger'], rac['Total Jobs'])
rac['Age%:Workers 30 to 54'] = percent(rac['Age:Workers 30 to 54'], rac['Total Jobs'])
rac['Age%:Workers 55 Older'] = percent(rac['Age:Workers 55 Older'], rac['Total Jobs'])
rac['Earnings%:1250mo or Less'] = percent(rac['Earnings:1250mo or Less'], rac['Total Jobs'])
rac['Earnings%:1251 to 3333mo'] = percent(rac['Earnings:1251 to 3333mo'], rac['Total Jobs'])
rac['Earnings%:Greater Than 3333mo'] = percent(rac['Earnings:Greater Than 3333mo'], rac['Total Jobs'])
rac['NAICS%:11Agriculture, Forestry, Fishing & Hunting'] = percent(rac['NAICS:11Agriculture, Forestry, Fishing & Hunting'], rac['Total Jobs'])
rac['NAICS%:21Mining, Quarrying, Oil & Gas'] = percent(rac['NAICS:21Mining, Quarrying, Oil & Gas'], rac['Total Jobs'])
rac['NAICS%:22Utilities'] = percent(rac['NAICS:22Utilities'], rac['Total Jobs'])
rac['NAICS%:23Construction'] = percent(rac['NAICS:23Construction'], rac['Total Jobs'])
rac['NAICS%:31to33Manufacturing'] = percent(rac['NAICS:31to33Manufacturing'], rac['Total Jobs'])
rac['NAICS%:42Wholesale Trade'] = percent(rac['NAICS:42Wholesale Trade'], rac['Total Jobs'])
rac['NAICS%:44to45Retail Trade'] = percent(rac['NAICS:44to45Retail Trade'], rac['Total Jobs'])
rac['NAICS%:48to49Transportation & Warehousing'] = percent(rac['NAICS:48to49Transportation & Warehousing'], rac['Total Jobs'])
rac['NAICS%:51Information'] = percent(rac['NAICS:51Information'], rac['Total Jobs'])
rac['NAICS%:52Finance & Insurance'] = percent(rac['NAICS:52Finance & Insurance'], rac['Total Jobs'])
rac['NAICS%:48to49Transportation & Warehousing'] = percent(rac['NAICS:48to49Transportation & Warehousing'], rac['Total Jobs'])
rac['NAICS%:51Information'] = percent(rac['NAICS:51Information'], rac['Total Jobs'])
rac['NAICS%:52Finance & Insurance'] = percent(rac['NAICS:52Finance & Insurance'], rac['Total Jobs'])
rac['NAICS%:53Real Estate & Rental & Leasing'] = percent(rac['NAICS:53Real Estate & Rental & Leasing'], rac['Total Jobs'])
rac['NAICS%:54Professional, Scientific, & Technical Services'] = percent(rac['NAICS:54Professional, Scientific, & Technical Services'], rac['Total Jobs'])
rac['NAICS%:55Management of Companies & Enterprises'] = percent(rac['NAICS:55Management of Companies & Enterprises'], rac['Total Jobs'])
rac['NAICS%:56Administrative & Support & Waste Management & Remediation Services'] = percent(rac['NAICS:56Administrative & Support & Waste Management & Remediation Services'], 
                                                                                             rac['Total Jobs'])
rac['NAICS%:61Educational Services'] = percent(rac['NAICS:61Educational Services'], rac['Total Jobs'])
rac['NAICS%:62Healthcare & Social Assistance'] = percent(rac['NAICS:62Healthcare & Social Assistance'], rac['Total Jobs'])
rac['NAICS%:71Arts, Entertainment, & Recreation'] = percent(rac['NAICS:71Arts, Entertainment, & Recreation'], rac['Total Jobs'])
rac['NAICS%:72Accommodation & Food Services'] = percent(rac['NAICS:72Accommodation & Food Services'], rac['Total Jobs'])
rac['NAICS%:81Other Services (Except Public Administration)'] = percent(rac['NAICS:81Other Services (Except Public Administration)'], rac['Total Jobs'])
rac['NAICS%:92Public Administration'] = percent(rac['NAICS:92Public Administration'], rac['Total Jobs'])
rac['Race%:White Alone'] = percent(rac['Race:White Alone'], rac['Total Jobs'])
rac['Race%:Black or African American Alone'] = percent(rac['Race:Black or African American Alone'], rac['Total Jobs'])
rac['Race%:American Indian or Alaska Native Alone'] = percent(rac['Race:American Indian or Alaska Native Alone'], rac['Total Jobs'])
rac['Race%:Asian Alone'] = percent(rac['Race:Asian Alone'], rac['Total Jobs'])
rac['Race%:Native Hawaiian or Other Pacific Islander Alone'] = percent(rac['Race:Native Hawaiian or Other Pacific Islander Alone'], rac['Total Jobs'])
rac['Race%:Two or More Races'] = percent(rac['Race:Two or More Races'], rac['Total Jobs'])
rac['Ethnicity%:Not Hispanic or Latino'] = percent(rac['Ethnicity:Not Hispanic or Latino'], rac['Total Jobs'])
rac['Ethnicity%:Hispanic or Latino'] = percent(rac['Ethnicity:Hispanic or Latino'], rac['Total Jobs'])
rac['Ed%:Less than High School'] = percent(rac['Ed:Less than High School'], rac['Total Jobs'])
rac['Ed%:High School or Equivalent'] = percent(rac['Ed:High School or Equivalent'], rac['Total Jobs'])
rac['Ed%:Some College or Associates'] = percent(rac['Ed:Some College or Associates'], rac['Total Jobs'])
rac['Ed%:Bachelors or Advanced Degree'] = percent(rac['Ed:Bachelors or Advanced Degree'], rac['Total Jobs'])
rac['Sex%:Male'] = percent(rac['Sex:Male'], rac['Total Jobs'])
rac['Sex%:Female'] = percent(rac['Sex:Female'], rac['Total Jobs'])

In [142]:
rac = rac.set_index('GEO_ID')
rac = rac.add_prefix('RAC:')

In [143]:
rac.head()

,RAC:Total Jobs,RAC:Age:Workers 29 or Younger,RAC:Age:Workers 30 to 54,RAC:Age:Workers 55 Older,RAC:Earnings:1250mo or Less,RAC:Earnings:1251 to 3333mo,RAC:Earnings:Greater Than 3333mo,"RAC:NAICS:11Agriculture, Forestry, Fishing & Hunting","RAC:NAICS:21Mining, Quarrying, Oil & Gas",RAC:NAICS:22Utilities,RAC:NAICS:23Construction,RAC:NAICS:31to33Manufacturing,RAC:NAICS:42Wholesale Trade,RAC:NAICS:44to45Retail Trade,RAC:NAICS:48to49Transportation & Warehousing,RAC:NAICS:51Information,RAC:NAICS:52Finance & Insurance,RAC:NAICS:53Real Estate & Rental & Leasing,"RAC:NAICS:54Professional, Scientific, & Technical Services",RAC:NAICS:55Management of Companies & Enterprises,RAC:NAICS:56Administrative & Support & Waste Management & Remediation Services,RAC:NAICS:61Educational Services,RAC:NAICS:62Healthcare & Social Assistance,"RAC:NAICS:71Arts, Entertainment, & Recreation",RAC:NAICS:72Accommodation & Food Services,RAC:NAICS:81Other Services (Except Public Administration),RAC:NAICS:92Public Administration,RAC:Race:White Alone,RAC:Race:Black or African American Alone,RAC:Race:American Indian or Alaska Native Alone,RAC:Race:Asian Alone,RAC:Race:Native Hawaiian or Other Pacific Islander Alone,RAC:Race:Two or More Races,RAC:Ethnicity:Not Hispanic or Latino,RAC:Ethnicity:Hispanic or Latino,RAC:Ed:Less than High School,RAC:Ed:High School or Equivalent,RAC:Ed:Some College or Associates,RAC:Ed:Bachelors or Advanced Degree,RAC:Sex:Male,RAC:Sex:Female,RAC:createdate,RAC:Age%:Workers 29 or Younger,RAC:Age%:Workers 30 to 54,RAC:Age%:Workers 55 Older,RAC:Earnings%:1250mo or Less,RAC:Earnings%:1251 to 3333mo,RAC:Earnings%:Greater Than 3333mo,"RAC:NAICS%:11Agriculture, Forestry, Fishing & Hunting","RAC:NAICS%:21Mining, Quarrying, Oil & Gas",RAC:NAICS%:22Utilities,RAC:NAICS%:23Construction,RAC:NAICS%:31to33Manufacturing,RAC:NAICS%:42Wholesale Trade,RAC:NAICS%:44to45Retail Trade,RAC:NAICS%:48to49Transportation & Warehousing,RAC:NAICS%:51Information,RAC:NAICS%:52Finance & Insurance,RAC:NAICS%:53Real Estate & Rental & Leasing,"RAC:NAICS%:54Professional, Scientific, & Technical Services",RAC:NAICS%:55Management of Companies & Enterprises,RAC:NAICS%:56Administrative & Support & Waste Management & Remediation Services,RAC:NAICS%:61Educational Services,RAC:NAICS%:62Healthcare & Social Assistance,"RAC:NAICS%:71Arts, Entertainment, & Recreation",RAC:NAICS%:72Accommodation & Food Services,RAC:NAICS%:81Other Services (Except Public Administration),RAC:NAICS%:92Public Administration,RAC:Race%:White Alone,RAC:Race%:Black or African American Alone,RAC:Race%:American Indian or Alaska Native Alone,RAC:Race%:Asian Alone,RAC:Race%:Native Hawaiian or Other Pacific Islander Alone,RAC:Race%:Two or More Races,RAC:Ethnicity%:Not Hispanic or Latino,RAC:Ethnicity%:Hispanic or Latino,RAC:Ed%:Less than High School,RAC:Ed%:High School or Equivalent,RAC:Ed%:Some College or Associates,RAC:Ed%:Bachelors or Advanced Degree,RAC:Sex%:Male,RAC:Sex%:Female
GEO_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
470010201001006,76,28,39,9,15,31,30,0,0,0,3,6,2,6,1,2,5,1,9,0,9,7,12,0,9,3,1,66,7,2,1,0,0,66,10,8,17,16,7,40,36,20211018,36.842105,51.315789,11.842105,19.736842,40.789474,39.473684,0.0,0.0,0.0,3.947368,7.894737,2.631579,7.894737,1.315789,2.631579,6.578947,1.315789,11.842105,0.000000,11.842105,9.210526,15.789474,0.000000,11.842105,3.947368,1.315789,86.842105,9.210526,2.631579,1.315789,0.0,0.000000,86.842105,13.157895,10.526316,22.368421,21.052632,9.210526,52.631579,47.368421
470010201001015,125,46,61,18,26,49,50,0,0,0,1,15,4,21,1,1,4,4,13,3,11,8,14,1,18,4,2,91,15,0,14,0,5,117,8,6,27,26,20,66,59,20211018,36.800000,48.800000,14.400000,20.800000,39.200000,40.000000,0.0,0.0,0.0,0.800000,12.000000,3.200000,16.800000,0.800000,0.800000,3.200000,3.200000,10.400000,2.400000,8.800000,6.400000,11.200000,0.800000,14.400000,3.200000,1.600000,72.800000,12.000000,0.000000,11.200000,0.0,4.000000,93.600000,6.400000,4.800000,21.600000,20.800000,16.000000,52.800000,47.2

In [144]:
#all primary jobs
url = 'https://lehd.ces.census.gov/data/lodes/LODES7/tn/wac/tn_wac_S000_JT01_2019.csv.gz'
wac = pd.read_csv(url)
print('Your file is ready.')

Your file is ready.


In [145]:
waccols = {'w_geocode': 'GEO_ID', 'C000': 'Total Jobs', 'CA01': 'Age:Workers 29 or Younger', 'CA02': 'Age:Workers 30 to 54', 
           'CA03': 'Age:Workers 55 Older', 'CE01': 'Earnings:1250mo or Less', 'CE02': 'Earnings:1251 to 3333mo', 
           'CE03': 'Earnings:Greater Than 3333mo', 'CNS01': 'NAICS:11Agriculture, Forestry, Fishing & Hunting', 
           'CNS02': 'NAICS:21Mining, Quarrying, Oil & Gas', 'CNS03': 'NAICS:22Utilities', 'CNS04': 'NAICS:23Construction', 'CNS05':'NAICS:31to33Manufacturing',
           'CNS06': 'NAICS:42Wholesale Trade', 'CNS07': 'NAICS:44to45Retail Trade', 'CNS08': 'NAICS:48to49Transportation & Warehousing',
           'CNS09': 'NAICS:51Information', 'CNS10': 'NAICS:52Finance & Insurance', 'CNS11': 'NAICS:53Real Estate & Rental & Leasing', 
           'CNS12': 'NAICS:54Professional, Scientific, & Technical Services', 'CNS13': 'NAICS:55Management of Companies & Enterprises', 
           'CNS14': 'NAICS:56Administrative & Support & Waste Management & Remediation Services', 'CNS15': 'NAICS:61Educational Services', 
           'CNS16': 'NAICS:62Healthcare & Social Assistance', 'CNS17': 'NAICS:71Arts, Entertainment, & Recreation', 'CNS18': 'NAICS:72Accommodation & Food Services', 
           'CNS19': 'NAICS:81Other Services (Except Public Administration)', 'CNS20': 'NAICS:92Public Administration', 
           'CR01': 'Race:White Alone', 'CR02': 'Race:Black or African American Alone', 'CR03': 'Race:American Indian or Alaska Native Alone', 
           'CR04': 'Race:Asian Alone', 'CR05': 'Race:Native Hawaiian or Other Pacific Islander Alone', 'CR07': 'Race:Two or More Races', 
           'CT01': 'Ethnicity:Not Hispanic or Latino', 'CT02': 'Ethnicity:Hispanic or Latino', 'CD01':'Ed:Less than High School', 
           'CD02': 'Ed:High School or Equivalent', 'CD03': 'Ed:Some College or Associates', 'CD04': 'Ed:Bachelors or Advanced Degree', 'CS01': 'Sex:Male', 
           'CS02': 'Sex:Female', 'CFA01': 'FirmAge:0 to 1 Year', 'CFA02': 'FirmAge:2 to 3 Years', 'CFA03': 'FirmAge:4 to 5 Years', 'CFA04': 'FirmAge:6 to 10 Years',
           'CFA05': 'FirmAge:11 or More Years', 'CFS01': 'FirmSize:0 to 19 Employees', 'CFS02': 'FirmSize:20 to 49 Employees', 
           'CFS03': 'FirmSize:50 to 249 Employees', 'CFS04': 'FirmSize:250 to 499 Employees', 'CFS05': 'FirmSize:500 or More Employees'}

In [146]:
wac.rename(columns = waccols, inplace = True)

In [147]:
wac['Age%:Workers 29 or Younger'] = percent(wac['Age:Workers 29 or Younger'], wac['Total Jobs'])
wac['Age%:Workers 30 to 54'] = percent(wac['Age:Workers 30 to 54'], wac['Total Jobs'])
wac['Age%:Workers 55 Older'] = percent(wac['Age:Workers 55 Older'], wac['Total Jobs'])
wac['Earnings%:1250mo or Less'] = percent(wac['Earnings:1250mo or Less'], wac['Total Jobs'])
wac['Earnings%:1251 to 3333mo'] = percent(wac['Earnings:1251 to 3333mo'], wac['Total Jobs'])
wac['Earnings%:Greater Than 3333mo'] = percent(wac['Earnings:Greater Than 3333mo'], wac['Total Jobs'])
wac['NAICS%:11Agriculture, Forestry, Fishing & Hunting'] = percent(wac['NAICS:11Agriculture, Forestry, Fishing & Hunting'], wac['Total Jobs'])
wac['NAICS%:21Mining, Quarrying, Oil & Gas'] = percent(wac['NAICS:21Mining, Quarrying, Oil & Gas'], wac['Total Jobs'])
wac['NAICS%:22Utilities'] = percent(wac['NAICS:22Utilities'], wac['Total Jobs'])
wac['NAICS%:23Construction'] = percent(wac['NAICS:23Construction'], wac['Total Jobs'])
wac['NAICS%:31to33Manufacturing'] = percent(wac['NAICS:31to33Manufacturing'], wac['Total Jobs'])
wac['NAICS%:42Wholesale Trade'] = percent(wac['NAICS:42Wholesale Trade'], wac['Total Jobs'])
wac['NAICS%:44to45Retail Trade'] = percent(wac['NAICS:44to45Retail Trade'], wac['Total Jobs'])
wac['NAICS%:48to49Transportation & Warehousing'] = percent(wac['NAICS:48to49Transportation & Warehousing'], wac['Total Jobs'])
wac['NAICS%:51Information'] = percent(wac['NAICS:51Information'], wac['Total Jobs'])
wac['NAICS%:52Finance & Insurance'] = percent(wac['NAICS:52Finance & Insurance'], wac['Total Jobs'])
wac['NAICS%:48to49Transportation & Warehousing'] = percent(wac['NAICS:48to49Transportation & Warehousing'], wac['Total Jobs'])
wac['NAICS%:51Information'] = percent(wac['NAICS:51Information'], wac['Total Jobs'])
wac['NAICS%:52Finance & Insurance'] = percent(wac['NAICS:52Finance & Insurance'], wac['Total Jobs'])
wac['NAICS%:53Real Estate & Rental & Leasing'] = percent(wac['NAICS:53Real Estate & Rental & Leasing'], wac['Total Jobs'])
wac['NAICS%:54Professional, Scientific, & Technical Services'] = percent(wac['NAICS:54Professional, Scientific, & Technical Services'], wac['Total Jobs'])
wac['NAICS%:55Management of Companies & Enterprises'] = percent(wac['NAICS:55Management of Companies & Enterprises'], wac['Total Jobs'])
wac['NAICS%:56Administrative & Support & Waste Management & Remediation Services'] = percent(wac['NAICS:56Administrative & Support & Waste Management & Remediation Services'], 
                                                                                             wac['Total Jobs'])
wac['NAICS%:61Educational Services'] = percent(wac['NAICS:61Educational Services'], wac['Total Jobs'])
wac['NAICS%:62Healthcare & Social Assistance'] = percent(wac['NAICS:62Healthcare & Social Assistance'], wac['Total Jobs'])
wac['NAICS%:71Arts, Entertainment, & Recreation'] = percent(wac['NAICS:71Arts, Entertainment, & Recreation'], wac['Total Jobs'])
wac['NAICS%:72Accommodation & Food Services'] = percent(wac['NAICS:72Accommodation & Food Services'], wac['Total Jobs'])
wac['NAICS%:81Other Services (Except Public Administration)'] = percent(wac['NAICS:81Other Services (Except Public Administration)'], wac['Total Jobs'])
wac['NAICS%:92Public Administration'] = percent(wac['NAICS:92Public Administration'], wac['Total Jobs'])
wac['Race%:White Alone'] = percent(wac['Race:White Alone'], wac['Total Jobs'])
wac['Race%:Black or African American Alone'] = percent(wac['Race:Black or African American Alone'], wac['Total Jobs'])
wac['Race%:American Indian or Alaska Native Alone'] = percent(wac['Race:American Indian or Alaska Native Alone'], wac['Total Jobs'])
wac['Race%:Asian Alone'] = percent(wac['Race:Asian Alone'], wac['Total Jobs'])
wac['Race%:Native Hawaiian or Other Pacific Islander Alone'] = percent(wac['Race:Native Hawaiian or Other Pacific Islander Alone'], wac['Total Jobs'])
wac['Race%:Two or More Races'] = percent(wac['Race:Two or More Races'], wac['Total Jobs'])
wac['Ethnicity%:Not Hispanic or Latino'] = percent(wac['Ethnicity:Not Hispanic or Latino'], wac['Total Jobs'])
wac['Ethnicity%:Hispanic or Latino'] = percent(wac['Ethnicity:Hispanic or Latino'], wac['Total Jobs'])
wac['Ed%:Less than High School'] = percent(wac['Ed:Less than High School'], wac['Total Jobs'])
wac['Ed%:High School or Equivalent'] = percent(wac['Ed:High School or Equivalent'], wac['Total Jobs'])
wac['Ed%:Some College or Associates'] = percent(wac['Ed:Some College or Associates'], wac['Total Jobs'])
wac['Ed%:Bachelors or Advanced Degree'] = percent(wac['Ed:Bachelors or Advanced Degree'], wac['Total Jobs'])
wac['Sex%:Male'] = percent(wac['Sex:Male'], wac['Total Jobs'])
wac['Sex%:Female'] = percent(wac['Sex:Female'], wac['Total Jobs'])
wac['FirmAge%:0 to 1 Year'] = percent(wac['FirmAge:0 to 1 Year'], wac['Total Jobs'])
wac['FirmAge%:2 to 3 Years'] = percent(wac['FirmAge:2 to 3 Years'], wac['Total Jobs'])
wac['FirmAge%:4 to 5 Years'] = percent(wac['FirmAge:4 to 5 Years'], wac['Total Jobs'])
wac['FirmAge%:6 to 10 Years'] = percent(wac['FirmAge:6 to 10 Years'], wac['Total Jobs'])
wac['FirmAge%:11 or More Years'] = percent(wac['FirmAge:11 or More Years'], wac['Total Jobs'])
wac['FirmSize%:0 to 19 Employees'] = percent(wac['FirmSize:0 to 19 Employees'], wac['Total Jobs'])
wac['FirmSize%:20 to 49 Employees'] = percent(wac['FirmSize:20 to 49 Employees'], wac['Total Jobs'])
wac['FirmSize%:50 to 249 Employees'] = percent(wac['FirmSize:50 to 249 Employees'], wac['Total Jobs'])
wac['FirmSize%:250 to 499 Employees'] = percent(wac['FirmSize:250 to 499 Employees'], wac['Total Jobs'])
wac['FirmSize%:500 or More Employees'] = percent(wac['FirmSize:500 or More Employees'], wac['Total Jobs'])

In [148]:
wac = wac.set_index('GEO_ID')
wac = wac.add_prefix('WAC:')

In [149]:
wac.head()

,WAC:Total Jobs,WAC:Age:Workers 29 or Younger,WAC:Age:Workers 30 to 54,WAC:Age:Workers 55 Older,WAC:Earnings:1250mo or Less,WAC:Earnings:1251 to 3333mo,WAC:Earnings:Greater Than 3333mo,"WAC:NAICS:11Agriculture, Forestry, Fishing & Hunting","WAC:NAICS:21Mining, Quarrying, Oil & Gas",WAC:NAICS:22Utilities,WAC:NAICS:23Construction,WAC:NAICS:31to33Manufacturing,WAC:NAICS:42Wholesale Trade,WAC:NAICS:44to45Retail Trade,WAC:NAICS:48to49Transportation & Warehousing,WAC:NAICS:51Information,WAC:NAICS:52Finance & Insurance,WAC:NAICS:53Real Estate & Rental & Leasing,"WAC:NAICS:54Professional, Scientific, & Technical Services",WAC:NAICS:55Management of Companies & Enterprises,WAC:NAICS:56Administrative & Support & Waste Management & Remediation Services,WAC:NAICS:61Educational Services,WAC:NAICS:62Healthcare & Social Assistance,"WAC:NAICS:71Arts, Entertainment, & Recreation",WAC:NAICS:72Accommodation & Food Services,WAC:NAICS:81Other Services (Except Public Administration),WAC:NAICS:92Public Administration,WAC:Race:White Alone,WAC:Race:Black or African American Alone,WAC:Race:American Indian or Alaska Native Alone,WAC:Race:Asian Alone,WAC:Race:Native Hawaiian or Other Pacific Islander Alone,WAC:Race:Two or More Races,WAC:Ethnicity:Not Hispanic or Latino,WAC:Ethnicity:Hispanic or Latino,WAC:Ed:Less than High School,WAC:Ed:High School or Equivalent,WAC:Ed:Some College or Associates,WAC:Ed:Bachelors or Advanced Degree,WAC:Sex:Male,WAC:Sex:Female,WAC:FirmAge:0 to 1 Year,WAC:FirmAge:2 to 3 Years,WAC:FirmAge:4 to 5 Years,WAC:FirmAge:6 to 10 Years,WAC:FirmAge:11 or More Years,WAC:FirmSize:0 to 19 Employees,WAC:FirmSize:20 to 49 Employees,WAC:FirmSize:50 to 249 Employees,WAC:FirmSize:250 to 499 Employees,WAC:FirmSize:500 or More Employees,WAC:createdate,WAC:Age%:Workers 29 or Younger,WAC:Age%:Workers 30 to 54,WAC:Age%:Workers 55 Older,WAC:Earnings%:1250mo or Less,WAC:Earnings%:1251 to 3333mo,WAC:Earnings%:Greater Than 3333mo,"WAC:NAICS%:11Agriculture, Forestry, Fishing & Hunting","WAC:NAICS%:21Mining, Quarrying, Oil & Gas",WAC:NAICS%:22Utilities,WAC:NAICS%:23Construction,WAC:NAICS%:31to33Manufacturing,WAC:NAICS%:42Wholesale Trade,WAC:NAICS%:44to45Retail Trade,WAC:NAICS%:48to49Transportation & Warehousing,WAC:NAICS%:51Information,WAC:NAICS%:52Finance & Insurance,WAC:NAICS%:53Real Estate & Rental & Leasing,"WAC:NAICS%:54Professional, Scientific, & Technical Services",WAC:NAICS%:55Management of Companies & Enterprises,WAC:NAICS%:56Administrative & Support & Waste Management & Remediation Services,WAC:NAICS%:61Educational Services,WAC:NAICS%:62Healthcare & Social Assistance,"WAC:NAICS%:71Arts, Entertainment, & Recreation",WAC:NAICS%:72Accommodation & Food Services,WAC:NAICS%:81Other Services (Except Public Administration),WAC:NAICS%:92Public Administration,WAC:Race%:White Alone,WAC:Race%:Black or African American Alone,WAC:Race%:American Indian or Alaska Native Alone,WAC:Race%:Asian Alone,WAC:Race%:Native Hawaiian or Other Pacific Islander Alone,WAC:Race%:Two or More Races,WAC:Ethnicity%:Not Hispanic or Latino,WAC:Ethnicity%:Hispanic or Latino,WAC:Ed%:Less than High School,WAC:Ed%:High School or Equivalent,WAC:Ed%:Some College or Associates,WAC:Ed%:Bachelors or Advanced Degree,WAC:Sex%:Male,WAC:Sex%:Female,WAC:FirmAge%:0 to 1 Year,WAC:FirmAge%:2 to 3 Years,WAC:FirmAge%:4 to 5 Years,WAC:FirmAge%:6 to 10 Years,WAC:FirmAge%:11 or More Years,WAC:FirmSize%:0 to 19 Employees,WAC:FirmSize%:20 to 49 Employees,WAC:FirmSize%:50 to 249 Employees,WAC:FirmSize%:250 to 499 Employees,WAC:FirmSize%:500 or More Employees
GEO_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
470010201001000,97,35,42,20,30,48,19,0,0,0,1,0,0,11,0,0,21,0,1,0,0,0,16,11,16,20,0,91,3,0,1,0,2,93,4,7,19,23,13,29,68,0,0,0,0,0,0,0,0,0,0,20211018,36.082474,43.298969,20.618557,30.927835,49.484536,19.587629,0.0,0.0,0.0,1.030928,0.0,0.0,11.340206,0.0,0.0,21.649485,0.0,1.030928,0.0,0.0,0.0,16.494845,11.340206,16.494845,20.618557,0.0,93.814433,3.0

In [280]:
#all primary jobs
url = 'https://lehd.ces.census.gov/data/lodes/LODES7/tn/od/tn_od_main_JT01_2019.csv.gz'
od = pd.read_csv(url)
print('Your file is ready.')

Your file is ready.


In [281]:
odcols = {'h_geocode': 'HOMEGEO_ID', 'w_geocode': 'WORKGEO_ID', 'S000': 'Total Jobs', 'SA01': 'Age:Workers 29 or Younger', 'SA02': 'Age:Workers 30 to 54', 
          'SA03': 'Age:Workers 55 Older', 'SE01': 'Earnings:1250mo or Less', 'SE02': 'Earnings:1251 to 3333mo', 'SE03': 'Earnings:Greater Than 3333mo', 
          'SI01': 'Sector:Goods Producing', 'SI02': 'Sector:Trade, Transportation & Utilities', 'SI03': 'Sector:All Other Services'}

In [282]:
od.rename(columns = odcols, inplace = True)

In [283]:
od['Age%:Workers 29 or Younger'] = percent(od['Age:Workers 29 or Younger'], od['Total Jobs'])
od['Age%:Workers 30 to 54'] = percent(od['Age:Workers 30 to 54'], od['Total Jobs'])
od['Age%:Workers 55 Older'] = percent(od['Age:Workers 55 Older'], od['Total Jobs'])
od['Earnings%:1250mo or Less'] = percent(od['Earnings:1250mo or Less'], od['Total Jobs'])
od['Earnings%:1251 to 3333mo'] = percent(od['Earnings:1251 to 3333mo'], od['Total Jobs'])
od['Earnings%:Greater Than 3333mo'] = percent(od['Earnings:Greater Than 3333mo'], od['Total Jobs'])
od['Sector%:Goods Producing'] = percent(od['Sector:Goods Producing'], od['Total Jobs'])
od['Sector%:Trade, Transportation & Utilities'] = percent(od['Sector:Trade, Transportation & Utilities'], od['Total Jobs'])
od['Sector%:All Other Services'] = percent(od['Sector:All Other Services'], od['Total Jobs'])

In [284]:
od = od.set_index(['HOMEGEO_ID', 'WORKGEO_ID'])
od = od.add_prefix('OD:')

In [285]:
od.head()

,,OD:Total Jobs,OD:Age:Workers 29 or Younger,OD:Age:Workers 30 to 54,OD:Age:Workers 55 Older,OD:Earnings:1250mo or Less,OD:Earnings:1251 to 3333mo,OD:Earnings:Greater Than 3333mo,OD:Sector:Goods Producing,"OD:Sector:Trade, Transportation & Utilities",OD:Sector:All Other Services,OD:createdate,OD:Age%:Workers 29 or Younger,OD:Age%:Workers 30 to 54,OD:Age%:Workers 55 Older,OD:Earnings%:1250mo or Less,OD:Earnings%:1251 to 3333mo,OD:Earnings%:Greater Than 3333mo,OD:Sector%:Goods Producing,"OD:Sector%:Trade, Transportation & Utilities",OD:Sector%:All Other Services
HOMEGEO_ID,WORKGEO_ID,,,,,,,,,,,,,,,,,,,,
470010201001063,470010201001000,2,1,1,0,0,1,1,0,0,2,20211018,50.0,50.0,0.0,0.0,50.0,50.0,0.0,0.0,100.0
470010201002013,470010201001000,1,0,0,1,1,0,0,0,0,1,20211018,0.0,0.0,100.0,100.0,0.0,0.0,0.0,0.0,100.0
470010201002017,470010201001000,1,1,0,0,0,1,0,0,0,1,20211018,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,100.0
470010201002021,470010201001000,1,1,0,0,0,1,0,0,0,1,20211018,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,100.0
470010202023018,470010201001000,1,0,0,1,0,0,1,0,0,1,20211018,0.0,0.0,100.0,0.0,0.0,100.0,0.0,0.0,100.0


In [286]:
od = od.reset_index(drop = False)

In [287]:
od['HOMEGEO_ID'] = od['HOMEGEO_ID'].astype(str)

In [288]:
test = od
test.head()

,HOMEGEO_ID,WORKGEO_ID,OD:Total Jobs,OD:Age:Workers 29 or Younger,OD:Age:Workers 30 to 54,OD:Age:Workers 55 Older,OD:Earnings:1250mo or Less,OD:Earnings:1251 to 3333mo,OD:Earnings:Greater Than 3333mo,OD:Sector:Goods Producing,"OD:Sector:Trade, Transportation & Utilities",OD:Sector:All Other Services,OD:createdate,OD:Age%:Workers 29 or Younger,OD:Age%:Workers 30 to 54,OD:Age%:Workers 55 Older,OD:Earnings%:1250mo or Less,OD:Earnings%:1251 to 3333mo,OD:Earnings%:Greater Than 3333mo,OD:Sector%:Goods Producing,"OD:Sector%:Trade, Transportation & Utilities",OD:Sector%:All Other Services
0,470010201001063,470010201001000,2,1,1,0,0,1,1,0,0,2,20211018,50.0,50.0,0.0,0.0,50.0,50.0,0.0,0.0,100.0
1,470010201002013,470010201001000,1,0,0,1,1,0,0,0,0,1,20211018,0.0,0.0,100.0,100.0,0.0,0.0,0.0,0.0,100.0
2,470010201002017,470010201001000,1,1,0,0,0,1,0,0,0,1,20211018,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,100.0
3,470010201002021,470010201001000,1,1,0,0,0,1,0,0,0,1,20211018,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,100.0
4,470010202023018,470010201001000,1,0,0,1,0,0,1,0,0,1,20211018,0.0,0.0,100.0,0.0,0.0,100.0,0.0,0.0,100.0


In [259]:
test = test.replace(new_dic)

In [260]:
test.head()

,HOMEGEO_ID,WORKGEO_ID,OD:Total Jobs,OD:Age:Workers 29 or Younger,OD:Age:Workers 30 to 54,OD:Age:Workers 55 Older,OD:Earnings:1250mo or Less,OD:Earnings:1251 to 3333mo,OD:Earnings:Greater Than 3333mo,OD:Sector:Goods Producing,"OD:Sector:Trade, Transportation & Utilities",OD:Sector:All Other Services,OD:createdate,OD:Age%:Workers 29 or Younger,OD:Age%:Workers 30 to 54,OD:Age%:Workers 55 Older,OD:Earnings%:1250mo or Less,OD:Earnings%:1251 to 3333mo,OD:Earnings%:Greater Than 3333mo,OD:Sector%:Goods Producing,"OD:Sector%:Trade, Transportation & Utilities",OD:Sector%:All Other Services
0,470010201001063,470010201001000,2,1,1,0,0,1,1,0,0,2,20211018,50.0,50.0,0.0,0.0,50.0,50.0,0.0,0.0,100.0
1,470010201002013,470010201001000,1,0,0,1,1,0,0,0,0,1,20211018,0.0,0.0,100.0,100.0,0.0,0.0,0.0,0.0,100.0
2,470010201002017,470010201001000,1,1,0,0,0,1,0,0,0,1,20211018,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,100.0
3,470010201002021,470010201001000,1,1,0,0,0,1,0,0,0,1,20211018,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,100.0
4,470010202023018,470010201001000,1,0,0,1,0,0,1,0,0,1,20211018,0.0,0.0,100.0,0.0,0.0,100.0,0.0,0.0,100.0


Geography Crosswalk

In [168]:
tncounty = ['47161', #Stewart
            '47125', #Montgomery
            '47083', #Houston
            '47085', #Humphreys
            '47043', #Dickson
            '47021', #Cheatham
            '47147', #Robertson
            '47165', #Sumner
            '47037', #Davidson
            '47189', #Wilson
            '47169', #Trousdale
            '47187', #Williamson
            '47149', #Rutherford
            '47119'] #Maury

In [169]:
url = 'https://lehd.ces.census.gov/data/lodes/LODES7/tn/tn_xwalk.csv.gz'
geo = pd.read_csv(url, dtype = str)
print('Your file is ready.')

Your file is ready.


In [170]:
#
geocols = {'tabblk2010': 'Block Code', 'st': 'StateFIPS', 'stusps': 'StateUSPS', 'stname': 'State', 'cty': 'CountyFIPS', 'ctyname': 'County', 
           'trct': 'TractCODE', 'trctname': 'Tract', 'bgrp': 'BlockgroupCODE', 'bgrpname': 'Blockgroup', 'cbsa': 'CBSACODE', 'cbsaname': 'CBSA', 
           'zcta': 'ZCTACODE', 'zctaname': 'ZCTA', 'stplc': 'PlaceCODE', 'stplcname': 'Place', 'stwib': 'WIBCODE', 'stwibname': 'WIB', 
           'blklatdd': 'Lat', 'blklondd': 'Lon'}
geo.rename(columns = geocols, inplace = True)

In [171]:
#drop unneeded columns
cols = ['ctycsub', 'ctycsubname', 'stcd116', 'stcd116name', 'stsldl', 'stsldlname', 'stsldu', 'stslduname', 'stschool',
       'stschoolname', 'stsecon', 'stseconname', 'trib', 'tribname', 'tsub', 'tsubname', 'stanrc', 'stanrcname', 'necta', 'nectaname', 'mil',
       'milname']
geo = geo.drop(columns = cols)

In [172]:
#make a list of block codes in the GNRC region
bg = geo.loc[geo['CountyFIPS'].isin(tncounty)]
blockcodegnrcregion = list(bg['Block Code'].values)
#blockcodegnrcregion

Create lists for counties and places.

In [207]:
#list of unique county names
tncounties = list(geo['County'].unique())
#tncounties

In [250]:
#create dictionary of county name to list of block codes
c = {}
for i in tncounties:
    c['{}'.format(i)] = list(geo.loc[geo['County'] == '{}'.format(i), 'Block Code'])
#c = pd.DataFrame.from_dict(c, orient = 'index').transpose()

In [251]:
c

{'Clay County, TN': ['470279550001069',
  '470279550001007',
  '470279550001001',
  '470279550001002',
  '470279550001057',
  '470279550001100',
  '470279550001050',
  '470279550001079',
  '470279550002003',
  '470279550002063',
  '470279550001028',
  '470279550002064',
  '470279550002061',
  '470279550001110',
  '470279551002067',
  '470279551002074',
  '470279551002071',
  '470279551002111',
  '470279551002037',
  '470279551002086',
  '470279551002068',
  '470279551002069',
  '470279551002051',
  '470279551002023',
  '470279551002021',
  '470279551002072',
  '470279551002036',
  '470279551002038',
  '470279551002052',
  '470279551002017',
  '470279551002070',
  '470279551002022',
  '470279551002042',
  '470279551002004',
  '470279551002043',
  '470279551002106',
  '470279551002044',
  '470279551002013',
  '470279551002001',
  '470279551002046',
  '470279551002039',
  '470279551002045',
  '470279551002041',
  '470279551002010',
  '470279551002110',
  '470279551002012',
  '470279551002

In [254]:
#invert that dictionary
new_dic = {}
for k,v in c.items():
    for x in v:
        new_dic.setdefault(x,[]).append(k)

In [269]:
new_dic

{'470279550001069': ['Clay County, TN'],
 '470279550001007': ['Clay County, TN'],
 '470279550001001': ['Clay County, TN'],
 '470279550001002': ['Clay County, TN'],
 '470279550001057': ['Clay County, TN'],
 '470279550001100': ['Clay County, TN'],
 '470279550001050': ['Clay County, TN'],
 '470279550001079': ['Clay County, TN'],
 '470279550002003': ['Clay County, TN'],
 '470279550002063': ['Clay County, TN'],
 '470279550001028': ['Clay County, TN'],
 '470279550002064': ['Clay County, TN'],
 '470279550002061': ['Clay County, TN'],
 '470279550001110': ['Clay County, TN'],
 '470279551002067': ['Clay County, TN'],
 '470279551002074': ['Clay County, TN'],
 '470279551002071': ['Clay County, TN'],
 '470279551002111': ['Clay County, TN'],
 '470279551002037': ['Clay County, TN'],
 '470279551002086': ['Clay County, TN'],
 '470279551002068': ['Clay County, TN'],
 '470279551002069': ['Clay County, TN'],
 '470279551002051': ['Clay County, TN'],
 '470279551002023': ['Clay County, TN'],
 '47027955100202

In [294]:
od['HOME County'] = od['HOMEGEO_ID'].map(new_dic)

In [293]:
od.head()

,HOMEGEO_ID,WORKGEO_ID,OD:Total Jobs,OD:Age:Workers 29 or Younger,OD:Age:Workers 30 to 54,OD:Age:Workers 55 Older,OD:Earnings:1250mo or Less,OD:Earnings:1251 to 3333mo,OD:Earnings:Greater Than 3333mo,OD:Sector:Goods Producing,"OD:Sector:Trade, Transportation & Utilities",OD:Sector:All Other Services,OD:createdate,OD:Age%:Workers 29 or Younger,OD:Age%:Workers 30 to 54,OD:Age%:Workers 55 Older,OD:Earnings%:1250mo or Less,OD:Earnings%:1251 to 3333mo,OD:Earnings%:Greater Than 3333mo,OD:Sector%:Goods Producing,"OD:Sector%:Trade, Transportation & Utilities",OD:Sector%:All Other Services,HOMECounty
0,470010201001063,470010201001000,2,1,1,0,0,1,1,0,0,2,20211018,50.0,50.0,0.0,0.0,50.0,50.0,0.0,0.0,100.0,"[Anderson County, TN]"
1,470010201002013,470010201001000,1,0,0,1,1,0,0,0,0,1,20211018,0.0,0.0,100.0,100.0,0.0,0.0,0.0,0.0,100.0,"[Anderson County, TN]"
2,470010201002017,470010201001000,1,1,0,0,0,1,0,0,0,1,20211018,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,100.0,"[Anderson County, TN]"
3,470010201002021,470010201001000,1,1,0,0,0,1,0,0,0,1,20211018,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,100.0,"[Anderson County, TN]"
4,470010202023018,470010201001000,1,0,0,1,0,0,1,0,0,1,20211018,0.0,0.0,100.0,0.0,0.0,100.0,0.0,0.0,100.0,"[Anderson County, TN]"


In [221]:
tnplace = geo[~geo['Place'].str.contains('CDP', na = True)]
tnplace = list(tnplace['Place'].unique())
tnplace

['Surgoinsville town, TN',
 'Franklin city, TN',
 'Milledgeville town, TN',
 'Enville town, TN',
 'Alcoa city, TN',
 'Maryville city, TN',
 'Louisville city, TN',
 'Caryville town, TN',
 'La Follette city, TN',
 'Jacksboro town, TN',
 'Elizabethton city, TN',
 'Tullahoma city, TN',
 'Manchester city, TN',
 'Nashville-Davidson metropolitan government (balance), TN',
 'Decherd city, TN',
 'Milan city, TN',
 'Church Hill city, TN',
 'Kingsport city, TN',
 'Mount Carmel town, TN',
 'Clarksville city, TN',
 'Smyrna town, TN',
 'Murfreesboro city, TN',
 'La Vergne city, TN',
 'Oneida town, TN',
 'Millington city, TN',
 'Farragut town, TN',
 'Knoxville city, TN',
 'Jamestown city, TN',
 'Allardt city, TN',
 'Auburntown town, TN',
 'Celina city, TN',
 'Woodbury town, TN',
 'South Carthage town, TN',
 'Gordonsville town, TN',
 'Red Boiling Springs city, TN',
 'Monterey town, TN',
 'Livingston town, TN',
 'Sparta city, TN',
 'Byrdstown town, TN',
 'Liberty town, TN',
 'Carthage town, TN',
 'Gain

In [243]:
p = {}
for i in tnplace:
    p['{}'.format(i)] = list(geo.loc[geo['Place'] == '{}'.format(i), 'Block Code'])
p = pd.DataFrame.from_dict(p, orient = 'index').transpose()

In [244]:
p.head()

,"Surgoinsville town, TN","Franklin city, TN","Milledgeville town, TN","Enville town, TN","Alcoa city, TN","Maryville city, TN","Louisville city, TN","Caryville town, TN","La Follette city, TN","Jacksboro town, TN","Elizabethton city, TN","Tullahoma city, TN","Manchester city, TN","Nashville-Davidson metropolitan government (balance), TN","Decherd city, TN","Milan city, TN","Church Hill city, TN","Kingsport city, TN","Mount Carmel town, TN","Clarksville city, TN","Smyrna town, TN","Murfreesboro city, TN","La Vergne city, TN","Oneida town, TN","Millington city, TN","Farragut town, TN","Knoxville city, TN","Jamestown city, TN","Allardt city, TN","Auburntown town, TN","Celina city, TN","Woodbury town, TN","South Carthage town, TN","Gordonsville town, TN","Red Boiling Springs city, TN","Monterey town, TN","Livingston town, TN","Sparta city, TN","Byrdstown town, TN","Liberty town, TN","Carthage town, TN","Gainesboro town, TN","Baxter town, TN","Smithville city, TN","Alexandria town, TN","Lafayette city, TN","Doyle town, TN","Cookeville city, TN","Dowelltown town, TN","Algood city, TN","Spencer town, TN","Spring Hill city, TN","Columbia city, TN","Mount Pleasant city, TN","Centerville town, TN","Lewisburg city, TN","Hohenwald city, TN","Chapel Hill town, TN","Lobelville city, TN","Linden town, TN","Elkton city, TN","Ardmore city, TN","Loretto city, TN","Pulaski city, TN","Lawrenceburg city, TN","Ethridge town, TN","Minor Hill city, TN","St. Joseph city, TN","Lynnville town, TN","Cornersville town, TN","Waynesboro city, TN","Collinwood city, TN","Clifton city, TN","Petersburg town, TN","Lebanon city, TN","Mount Juliet city, TN","Hartsville/Trousdale County, TN","Watertown city, TN","Goodlettsville city, TN","Ridgetop city, TN","Forest Hills city, TN","Belle Meade city, TN","Oak Hill city, TN","Berry Hill city, TN","Eagleville city, TN","Bristol city, TN","Mountain City town, TN","Johnson City city, TN","Jonesborough town, TN","Watauga city, TN","Erwin town, TN","Unicoi town, TN","Bluff City city, TN","Palmer town, TN","Gruetli-Laager city, TN","Shelbyville city, TN","Normandy town, TN","Wartrace town, TN","Bell Buckle town, TN","Beersheba Springs town, TN","Altamont town, TN","Coalmont city, TN","Viola town, TN","Morrison town, TN","McMinnville city, TN","Lynchburg, Moore County metropolitan government, TN","Centertown town, TN","Huntland town, TN","Fayetteville city, TN","Winchester city, TN","Estill Springs town, TN","Tracy City town, TN","Monteagle town, TN","Cowan city, TN","Sardis town, TN","Henderson city, TN","Stanton town, TN","Martin city, TN","Henry town, TN","Sharon town, TN","Dresden town, TN","Cottage Grove town, TN","McKenzie city, TN","Greenfield city, TN","Huntingdon town, TN","Jackson city, TN","Three Way city, TN","Humboldt city, TN","Trezevant town, TN","Atwood town, TN","McLemoresville town, TN","Clarksburg town, TN","Lexington city, TN","Scotts Hill town, TN","Parker's Crossroads city, TN","Brownsville city, TN","Decaturville town, TN","Parsons city, TN","Paris city, TN","Gleason town, TN","Puryear city, TN","Hollow Rock town, TN","Big Sandy town, TN","Camden city, TN","Bruceton town, TN","Adamsville town, TN","Finger city, TN","Medon city, TN","Bethel Springs town, TN","Stantonville town, TN","Michie town, TN","Guys town, TN","Selmer town, TN","Ramer city, TN","Eastview town, TN","Saltillo town, TN","Crump city, TN","Savannah city, TN","Whiteville town, TN","Silerton town, TN","Toone town, TN","Grand Junction city, TN","Saulsbury town, TN","Middleton city, TN","Bolivar city, TN","Hickory Valley town, TN","Hornsby town, TN","Arlington town, TN","Bartlett city, TN","Memphis city, TN","Lakeland city, TN","Germantown city, TN","Collierville town, TN","Braden town, TN","La Grange town, TN","Somerville town, TN","Williston city, TN","Moscow city, TN","Gallaway city, TN","Oakland town, TN","Rossville town, TN","Piperton city, TN","Baileyton town, TN","Rogersville town, TN","Sneedville town, TN","Bulls Gap town, TN","Ta

In [185]:
geo.head(20)

,Block Code,StateFIPS,StateUSPS,State,CountyFIPS,County,TractCODE,Tract,BlockgroupCODE,Blockgroup,CBSACODE,CBSA,ZCTACODE,ZCTA,PlaceCODE,Place,WIBCODE,WIB,Lat,Lon,createdate
0,470279550001069,47,TN,Tennessee,47027,"Clay County, TN",47027955000,"9550 (Clay, TN)",470279550001,"1 (Tract 9550, Clay, TN)",99999,NaN,38551,38551,9999999,NaN,47LWIA07,07 Local Workforce Investment Area,36.5522497,-85.4519031,20211018
1,470059630002096,47,TN,Tennessee,47005,"Benton County, TN",47005963000,"9630 (Benton, TN)",470059630002,"2 (Tract 9630, Benton, TN)",99999,NaN,38221,38221,9999999,NaN,47LWIA11,11 Local Workforce Investment Area,36.2107573,-88.1373529,20211018
2,470059630002091,47,TN,Tennessee,47005,"Benton County, TN",47005963000,"9630 (Benton, TN)",470059630002,"2 (Tract 9630, Benton, TN)",99999,NaN,38221,38221,9999999,NaN,47LWIA11,11 Local Workforce Investment Area,36.1901840,-88.0977486,20211018
3,470059631001043,47,TN,Tennessee,47005,"Benton County, TN",47005963100,"9631 (Benton, TN)",470059631001,"1 (Tract 9631, Benton, TN)",99999,NaN,38320,38320,9999999,NaN,47LWIA11,11 Local Workforce Investment Area,36.0788357,-88.0445962,20211018
4,470059630001061,47,TN,Tennessee,47005,"Benton County, TN",47005963000,"9630 (Benton, TN)",470059630001,"1 (Tract 9630, Benton, TN)",99999,NaN,38221,38221,9999999,NaN,47LWIA11,11 Local Workforce Investment Area,36.3057522,-88.0285893,20211018
5,470059631001007,47,TN,Tennessee,47005,"Benton County, TN",47005963100,"9631 (Benton, TN)",470059631001,"1 (Tract 9631, Benton, TN)",99999,NaN,38320,38320,9999999,NaN,47LWIA11,11 Local Workforce Investment Area,36.1154421,-88.0391200,20211018
6,470059631001032,47,TN,Tennessee,47005,"Benton County, TN",47005963100,"9631 (Benton, TN)",470059631001,"1 (Tract 9631, Benton, TN)",99999,NaN,38320,38320,9999999,NaN,47LWIA11,11 Local Workforce Investment Area,36.0911287,-88.0713279,20211018
7,470279550001007,47,TN,Tennessee,47027,"Clay County, TN",47027955000,"9550 (Clay, TN)",470279550001,"1 (Tract 9550, Clay, TN)",99999,NaN,38551,38551,9999999,NaN,47LWIA07,07 Local Workforce Investment Area,36.6179044,-85.2924035,20211018
8,470179624002060,47,TN,Tennessee,47017,"Carroll County, TN",47017962400,"9624 (Carroll, TN)",470179624002,"2 (Tract 9624, Carroll, TN)",99999,NaN,38305,38305,9999999,NaN,47LWIA11,11 Local Workforce Investment Area,35.8088893,-88.6246026,20211018
9,470279550001001,47,TN,Tennessee,47027,"Clay County, TN",47027955000,"9550 (Clay, TN)",470279550001,"1 (Tract 9550, Clay, TN)",99999,NaN,38551,38551,9999999,NaN,47LWIA07,07 Local Workforce Investment Area,36.6194362,-85.3272410,20211018


### Join RAC and WAC for comparisons.

In [151]:
rac = rac.reset_index(drop = False)
rac = rac.loc[rac['GEO_ID'].isin(blockcodegnrcregion)]

In [153]:
wac = wac.reset_index(drop = False)
wac = wac.loc[wac['GEO_ID'].isin(blockcodegnrcregion)]

In [173]:
df = pd.merge(left = wac, right = rac, on = 'GEO_ID', how = 'outer')
df.head()

,GEO_ID,WAC:Total Jobs,WAC:Age:Workers 29 or Younger,WAC:Age:Workers 30 to 54,WAC:Age:Workers 55 Older,WAC:Earnings:1250mo or Less,WAC:Earnings:1251 to 3333mo,WAC:Earnings:Greater Than 3333mo,"WAC:NAICS:11Agriculture, Forestry, Fishing & Hunting","WAC:NAICS:21Mining, Quarrying, Oil & Gas",WAC:NAICS:22Utilities,WAC:NAICS:23Construction,WAC:NAICS:31to33Manufacturing,WAC:NAICS:42Wholesale Trade,WAC:NAICS:44to45Retail Trade,WAC:NAICS:48to49Transportation & Warehousing,WAC:NAICS:51Information,WAC:NAICS:52Finance & Insurance,WAC:NAICS:53Real Estate & Rental & Leasing,"WAC:NAICS:54Professional, Scientific, & Technical Services",WAC:NAICS:55Management of Companies & Enterprises,WAC:NAICS:56Administrative & Support & Waste Management & Remediation Services,WAC:NAICS:61Educational Services,WAC:NAICS:62Healthcare & Social Assistance,"WAC:NAICS:71Arts, Entertainment, & Recreation",WAC:NAICS:72Accommodation & Food Services,WAC:NAICS:81Other Services (Except Public Administration),WAC:NAICS:92Public Administration,WAC:Race:White Alone,WAC:Race:Black or African American Alone,WAC:Race:American Indian or Alaska Native Alone,WAC:Race:Asian Alone,WAC:Race:Native Hawaiian or Other Pacific Islander Alone,WAC:Race:Two or More Races,WAC:Ethnicity:Not Hispanic or Latino,WAC:Ethnicity:Hispanic or Latino,WAC:Ed:Less than High School,WAC:Ed:High School or Equivalent,WAC:Ed:Some College or Associates,WAC:Ed:Bachelors or Advanced Degree,WAC:Sex:Male,WAC:Sex:Female,WAC:FirmAge:0 to 1 Year,WAC:FirmAge:2 to 3 Years,WAC:FirmAge:4 to 5 Years,WAC:FirmAge:6 to 10 Years,WAC:FirmAge:11 or More Years,WAC:FirmSize:0 to 19 Employees,WAC:FirmSize:20 to 49 Employees,WAC:FirmSize:50 to 249 Employees,WAC:FirmSize:250 to 499 Employees,WAC:FirmSize:500 or More Employees,WAC:createdate,WAC:Age%:Workers 29 or Younger,WAC:Age%:Workers 30 to 54,WAC:Age%:Workers 55 Older,WAC:Earnings%:1250mo or Less,WAC:Earnings%:1251 to 3333mo,WAC:Earnings%:Greater Than 3333mo,"WAC:NAICS%:11Agriculture, Forestry, Fishing & Hunting","WAC:NAICS%:21Mining, Quarrying, Oil & Gas",WAC:NAICS%:22Utilities,WAC:NAICS%:23Construction,WAC:NAICS%:31to33Manufacturing,WAC:NAICS%:42Wholesale Trade,WAC:NAICS%:44to45Retail Trade,WAC:NAICS%:48to49Transportation & Warehousing,WAC:NAICS%:51Information,WAC:NAICS%:52Finance & Insurance,WAC:NAICS%:53Real Estate & Rental & Leasing,"WAC:NAICS%:54Professional, Scientific, & Technical Services",WAC:NAICS%:55Management of Companies & Enterprises,WAC:NAICS%:56Administrative & Support & Waste Management & Remediation Services,WAC:NAICS%:61Educational Services,WAC:NAICS%:62Healthcare & Social Assistance,"WAC:NAICS%:71Arts, Entertainment, & Recreation",WAC:NAICS%:72Accommodation & Food Services,WAC:NAICS%:81Other Services (Except Public Administration),WAC:NAICS%:92Public Administration,WAC:Race%:White Alone,WAC:Race%:Black or African American Alone,WAC:Race%:American Indian or Alaska Native Alone,WAC:Race%:Asian Alone,WAC:Race%:Native Hawaiian or Other Pacific Islander Alone,WAC:Race%:Two or More Races,WAC:Ethnicity%:Not Hispanic or Latino,WAC:Ethnicity%:Hispanic or Latino,WAC:Ed%:Less than High School,WAC:Ed%:High School or Equivalent,WAC:Ed%:Some College or Associates,WAC:Ed%:Bachelors or Advanced Degree,WAC:Sex%:Male,WAC:Sex%:Female,WAC:FirmAge%:0 to 1 Year,WAC:FirmAge%:2 to 3 Years,WAC:FirmAge%:4 to 5 Years,WAC:FirmAge%:6 to 10 Years,WAC:FirmAge%:11 or More Years,WAC:FirmSize%:0 to 19 Employees,WAC:FirmSize%:20 to 49 Employees,WAC:FirmSize%:50 to 249 Employees,WAC:FirmSize%:250 to 499 Employees,WAC:FirmSize%:500 or More Employees,RAC:Total Jobs,RAC:Age:Workers 29 or Younger,RAC:Age:Workers 30 to 54,RAC:Age:Workers 55 Older,RAC:Earnings:1250mo or Less,RAC:Earnings:1251 to 3333mo,RAC:Earnings:Greater Than 3333mo,"RAC:NAICS:11Agriculture, Forestry, Fishing & Hunting","RAC:NAICS:21Mining, Quarrying, Oil & Gas",RAC:NAICS:22Utilities,RAC:NAICS:23Construction,RAC:NAICS:31to33Manufacturing,RAC:NAICS:42Wholesale Trade,RAC:NAICS:44to45Retail Trade,RAC:NAICS:48to49Transportation &